# Automated Resume Screening

### COMP 4750: Natural Language Processing
Shawon Ibn Kamal, 201761376



In [1]:
from pdfminer.high_level import extract_text
import docx2txt
import nltk
import re
import subprocess 

ModuleNotFoundError: No module named 'pdfminer'